In [ ]:
import sys
sys.path.append('../../../')

import torch
import numpy as np
from scipy.stats import multivariate_normal

from LRM.CMP1D import CMP1D, SmoothedEmpiricalDensity1D

from DiscreteFisherBayes.Source.Models import CMP
from DiscreteFisherBayes.Source.Posteriors import FDBayes, Bayes
import time

theta1 = 4.0
theta2 = 1.25
dnum = 3000
pnum = 5000
numboot = 100

RUN = True

In [2]:
#This part simulates data that we will use for each method to evaluate their cost.

if RUN:
    prior = torch.distributions.Chi2(torch.tensor([3.0, 3.0]))
    log_prior = lambda param: prior.log_prob(param).sum()
    transit_p = torch.distributions.Normal(torch.zeros(2), 0.1*torch.ones(2))
    
    #Run if needed only (it's 100k samples so takes a bit of time)
    #NM = 10000
    Ns = [250, 500, 750, 1000, 1250, 1500, 1750, 2000]
    NM = 100
    p0 = prior.sample()
    cmp = CMP()
    full_data = cmp.sample(torch.tensor([theta1, theta2]), 10*NM*100, 20000)[::100,:]

    datas = []
    print("Data...")
    for ith in range(len(Ns)):
        datas.append([])
        for jth in range(10):
            idxj = torch.randint(10*NM, (Ns[ith],))
            datas[ith].append(full_data[idxj,:])

Data...


# Bayes

In [ ]:
num_MCMC_samples = 5000
Ps = np.zeros((len(Ns), 10, num_MCMC_samples, 2))
Ts_P = np.zeros((len(Ns), 10))
Ts_P_total = np.zeros((len(Ns), 10))


if RUN:
    for args in ["Bayes"]:


        print("Computation...")
        for ith in range(len(Ns)):
            print(ith)

            for jth in range(10):
                data = datas[ith][jth]
                print("Dataset size: ", data.shape)
                
                time_start = time.time()
                
                beta_opt = torch.tensor([1.0])
                    
                time_mid = time.time()
                posterior = Bayes(cmp.uloglikelihood, torch.arange(500).reshape(500, 1), log_prior)
                posterior.set_X(data)
                post_sample_beta = posterior.sample(num_MCMC_samples, num_MCMC_samples, transit_p, prior.sample(), beta=beta_opt)
                time_end = time.time()

                Ts_P_total[ith, jth] = time_end - time_start
                Ts_P[ith, jth] = time_end - time_mid #just inference/MCMC times

                Ps[ith, jth, :, :] = post_sample_beta.numpy()
        
        print("Saving...")
        np.savez(f"./outputs/computation_{args}_theta1={theta1}_theta2={theta2}_numMCMCsamples={num_MCMC_samples}.npz", n=Ns, times_total=Ts_P_total, times=Ts_P, data=full_data)


Computation...
0
Dataset size:  torch.Size([250, 1])


100%|██████████| 5000/5000 [00:00<00:00, 5668.49it/s]


Dataset size:  torch.Size([250, 1])


100%|██████████| 5000/5000 [00:00<00:00, 6128.05it/s]


Dataset size:  torch.Size([250, 1])


...

Dataset size:  torch.Size([2000, 1])


100%|██████████| 5000/5000 [00:01<00:00, 4647.22it/s]


Dataset size:  torch.Size([2000, 1])


100%|██████████| 5000/5000 [00:01<00:00, 4693.99it/s]

Saving...


# DFD-Bayes 

In [ ]:
num_MCMC_samples = 5000
Ps = np.zeros((len(Ns), 1, num_MCMC_samples, 2))
Ts_P = np.zeros((len(Ns), 1))
Ts_P_total = np.zeros((len(Ns), 1))

def get_beta_opt(posterior, data):
        
    Ps = np.zeros((10, pnum, 2))
    beta_opt = torch.tensor([1.0])

    p_init, _ = posterior.minimise(posterior.loss, prior.sample(), ite=50000, lr=0.1, loss_thin=100, progress=False)
    boot_minimisers, _ = posterior.bootstrap_minimisers(data, numboot, lambda: p_init)
    posterior.set_X(data)
    beta_opt = posterior.optimal_beta(posterior.loss, boot_minimisers)
    return beta_opt


if RUN:
    for args in ["FDBayes"]:
    
        print("Computation...")
        for ith in range(len(Ns)):
            print(ith)

            for jth in range(1):
                data = datas[ith][jth]
                print("Dataset size: ", data.shape)
                                
                time_start = time.time()
                posterior = FDBayes(cmp.ratio_m, cmp.ratio_p, cmp.stat_m, cmp.stat_p, log_prior)
                posterior.set_X(data)

                beta_opt = get_beta_opt(posterior, data)
                    
                time_mid = time.time()
                post_sample_beta = posterior.sample(num_MCMC_samples, num_MCMC_samples, transit_p, prior.sample(), beta=beta_opt)
                time_end = time.time()

                Ts_P_total[ith, jth] = time_end - time_start
                Ts_P[ith, jth] = time_end - time_mid #just inference/MCMC times

                Ps[ith, jth, :, :] = post_sample_beta.numpy()
        
        print("Saving...")
        np.savez(f"./outputs/computation_{args}_theta1={theta1}_theta2={theta2}_numMCMCsamples={num_MCMC_samples}.npz", n=Ns, times_total=Ts_P_total, times=Ts_P, data=full_data)


Computation...
0
Dataset size:  torch.Size([250, 1])


100%|██████████| 5000/5000 [00:00<00:00, 7225.41it/s]


1
Dataset size:  torch.Size([500, 1])


100%|██████████| 5000/5000 [00:00<00:00, 6032.61it/s]


2
Dataset size:  torch.Size([750, 1])


...

6
Dataset size:  torch.Size([1750, 1])


100%|██████████| 5000/5000 [00:01<00:00, 4780.97it/s]


7
Dataset size:  torch.Size([2000, 1])


100%|██████████| 5000/5000 [00:01<00:00, 4939.27it/s]

Saving...


# LRM

In [4]:
def transform_theta_logtheta(mu_X, Sigma_X):

    mu_X = np.asarray(mu_X)
    Sigma_X = np.asarray(Sigma_X)

    theta1, theta2 = mu_X
    if theta1 <= 0:
        raise ValueError("theta1 must be positive for log transformation.")

    # g(mu_X)
    mu_Y = np.array([np.log(theta1), theta2])

    # Jacobian of g at mu_X
    J = np.array([
        [1/theta1, 0],
        [0,        1]
    ])

    # Covariance after transformation
    Sigma_Y = J @ Sigma_X @ J.T

    return mu_Y.reshape(-1,1), Sigma_Y

In [ ]:
prior = multivariate_normal(mean=[3.0, 3.0], cov=[[1.0, 0.0], [0.0, 1.0]])
prior_mean, prior_cov = transform_theta_logtheta(prior.mean, prior.cov)

Ps = np.zeros((len(Ns), 10, 5000, 2))
Ts_P = np.zeros((len(Ns), 10))
Ts_P_total = np.zeros((len(Ns), 10))

if RUN:
    for ith in range(len(Ns)):
        print(ith)

        for jth in range(10):
            data = datas[ith][jth]
            data = data.numpy().flatten()
            print("Dataset size: ", data.shape)
            
            
            time_start = time.time()
            empirical = SmoothedEmpiricalDensity1D(alpha=0.0)
            empirical.fit(data)
            cmp_lrm = CMP1D(empirical=empirical)
            beta, _, _ = cmp_lrm.fit_coverage(data=data, prior_mean=prior_mean, prior_cov=prior_cov, verbose=True, B=50)

            time_mid = time.time()
            posterior = cmp_lrm.posterior(data, beta=beta, mu_prior=prior_mean, Sigma_prior=prior_cov)

            time_end = time.time()

            Ts_P_total[ith, jth] = time_end - time_start
            Ts_P[ith, jth] = time_end - time_mid #just inference/MCMC times
    
    print("Saving...")
    np.savez(f"./outputs/computation_LRM_theta1={theta1}_theta2={theta2}.npz", n=Ns, times_total=Ts_P_total, times=Ts_P, data=full_data)


0
Dataset size:  (250,)
[scipy] beta*: 0.229144, coverage: 0.9600 (target 0.9500); fun=0.0001, success=True
Dataset size:  (250,)
[scipy] beta*: 0.557997, coverage: 0.9600 (target 0.9500); fun=0.0001, success=True
Dataset size:  (250,)
[scipy] beta*: 0.557759, coverage: 0.8200 (target 0.9500); fun=0.0169, success=True
Dataset size:  (250,)
[scipy] beta*: 0.215994, coverage: 0.9600 (target 0.9500); fun=0.0001, success=True
Dataset size:  (250,)
[scipy] beta*: 0.345047, coverage: 0.9600 (target 0.9500); fun=0.0001, success=True
Dataset size:  (250,)
[scipy] beta*: 0.413476, coverage: 0.9400 (target 0.9500); fun=0.0001, success=True
Dataset size:  (250,)
[scipy] beta*: 0.402504, coverage: 0.9800 (target 0.9500); fun=0.0009, success=True
Dataset size:  (250,)
[scipy] beta*: 0.657923, coverage: 0.2000 (target 0.9500); fun=0.5625, success=True
Dataset size:  (250,)
[scipy] beta*: 0.461253, coverage: 0.9600 (target 0.9500); fun=0.0001, success=True
Dataset size:  (250,)
[scipy] beta*: 0.55801